__<font size="6" weight="bold">Bayes Trainer for Validator</font>__
<br>
<strong>Not for production</strong>
Hier werden die Modelle für die SVM und den Bayes angelernt. <br>
Dazu werden alle Pattern durchlaufen. Für jedes Pattern wird ein DataFrame mit positiven und negativen Sätzen erstellt. <br>
In diesem Notebook werden nur die Ergebnisse des besten Bayes(siehe Dokumenttion) und der besten SVM(siehe Dokumentation) neu trainiert,beide werden mit den gleichen Sätzen trainiert, diese können sich jedoch zwischen Bays und SVM unterscheiden. Zum Beispiel sind beim besten Bayes die Stoppwörter entfernt worden, jedoch wurden die Sätze nicht lemmatisiert. 

## Initialisieren der benötigten Klassen

In [6]:
from sklearn.pipeline import Pipeline
import pickle
import os 
import os.path as path
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB


%run REST.ipynb
client = empamos_rest_client()
%run TrainingSetFactory_for_BayesTrainer_lemma_no_lemma_stopwords_no_stopwords.ipynb

## Test Tokenisieren mit Stoppwörter/Tokenisieren ohne Stoppwörter/Nicht Tokenisieren mit Stoppwörter/Nicht Tokenisieren ohne Stoppwörter


In [2]:
# Erstelle Ordner zur Ablage der Modelle und des Outputs
directoryname = "SVM_Bayes_comparison"
os.makedirs(directoryname, exist_ok=True)
directoryname_output = "SVM_Bayes_comparison/Output"
os.makedirs(directoryname_output, exist_ok=True)


In [7]:
factory = TrainingSetFactory(4)
pattern_ids = [pattern['pattern_id'] for pattern in client.get_patterns()]

for id in pattern_ids:
    print(id)
    df = factory.trainings_dataframe(id) 
    if (factory.isValid):
        #Dataframe for bayes
        dfPos = factory.Sentences_to_DF(factory.no_tokenized_removed_stopwords_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.no_tokenized_removed_stopwords_neg_sentences,0)
        df = dfPos.append(dfNeg)
        X=df[0]
        Y=df[1]
        X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.85, random_state=0)
        Bayes_Train(id,X_train, X_test, y_train, y_test,"without_Stopwords_no_lemmatize")
        #Dataframe for SVM
        dfPos = factory.Sentences_to_DF(factory.tokenized_stopwords_removed_pos_sentences,1)
        dfNeg = factory.Sentences_to_DF(factory.tokenized_stopwords_removed_neg_sentences,0)
        df = dfPos.append(dfNeg)
        X=df[0]
        Y=df[1]
        X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.85, random_state=0)
        SVM_Train(id,X_train, X_test, y_train, y_test,"stopwords_removed_lemmatize")
    else:
        print(f'Keine Daten vorhanden! {id}')


        
        
        

    

1292
1409
Keine Daten vorhanden! 1409
1417
Keine Daten vorhanden! 1417
1433
Keine Daten vorhanden! 1433
1446
Keine Daten vorhanden! 1446
1455
Keine Daten vorhanden! 1455
1464
Keine Daten vorhanden! 1464
1473
Keine Daten vorhanden! 1473
1482
Keine Daten vorhanden! 1482
1492
1515
Keine Daten vorhanden! 1515
1665
Keine Daten vorhanden! 1665
1756
Keine Daten vorhanden! 1756
2884
Keine Daten vorhanden! 2884
2894
Keine Daten vorhanden! 2894
2907
Keine Daten vorhanden! 2907
2968
Keine Daten vorhanden! 2968
2981
Keine Daten vorhanden! 2981
3085
Keine Daten vorhanden! 3085
3098
Keine Daten vorhanden! 3098
3142


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3173
3194
Keine Daten vorhanden! 3194
3285
Keine Daten vorhanden! 3285
3312
Keine Daten vorhanden! 3312
3377
Keine Daten vorhanden! 3377
3755
Keine Daten vorhanden! 3755
4426
Keine Daten vorhanden! 4426
5718
Keine Daten vorhanden! 5718
5729
Keine Daten vorhanden! 5729
5740
Keine Daten vorhanden! 5740
7090
7098
Keine Daten vorhanden! 7098
7106
Keine Daten vorhanden! 7106
7115
Keine Daten vorhanden! 7115
7123
Keine Daten vorhanden! 7123
7227
Keine Daten vorhanden! 7227
8520
Keine Daten vorhanden! 8520
8528
8536
8544
Keine Daten vorhanden! 8544
8552
Keine Daten vorhanden! 8552
8560
Keine Daten vorhanden! 8560
8568
Keine Daten vorhanden! 8568
8576
Keine Daten vorhanden! 8576
8584
Keine Daten vorhanden! 8584
8592
Keine Daten vorhanden! 8592
8608
Keine Daten vorhanden! 8608
8616
Keine Daten vorhanden! 8616
8624
Keine Daten vorhanden! 8624
8632
Keine Daten vorhanden! 8632
8640
Keine Daten vorhanden! 8640
8656
Keine Daten vorhanden! 8656
8664
Keine Daten vorhanden! 8664
8680
Keine Daten vorhan

In [3]:
def SVM_Train(id,X_train, X_test, y_train, y_test,msg=""):
    lsvc = Pipeline([('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', LinearSVC(random_state=None)),
       ])

    parameters = {
         "vect__ngram_range": [(1, 1), (1, 2), (1, 3), (1, 4)],
         "tfidf__use_idf": (True, False),
         "clf__loss" : ['hinge', 'squared_hinge'],
         "clf__tol" : [1e-6, 1e-5, 1e-4]
    }

    ##Hier zu betrachtendes Attribut über scoring = <attribute> wählen
    gs_clf = GridSearchCV(lsvc, parameters, cv=5, n_jobs=-1, scoring = 'accuracy')
    gs_clf = gs_clf.fit(X_train, y_train)
    #predit test
    predictions = gs_clf.predict(X_test)
    pickle.dump(gs_clf.best_estimator_,open(directoryname+"/SVM_Pattern_Model_"+msg+str(id),"wb"))
    #
    with open(directoryname_output+"/SVM_Pattern_Model_evaluation_"+msg+str(id),"w") as file:
            file.write("Accuracy score"+ str (accuracy_score(y_test, predictions))+"\n")
            file.write('Precision score: ' + str (precision_score(y_test, predictions,pos_label=0))+"\n")
            file.write('Recall score: ' + str (recall_score(y_test, predictions,pos_label=0))+"\n")
            file.write('f1_score score: ' + str (f1_score(y_test, predictions,pos_label=0))+"\n")

    
    

In [4]:
def Bayes_Train(id,X_train, X_test, y_train, y_test,msg=""):
        naive_bayes = BernoulliNB()
        #wordvetor erstellung
        vectorizer = CountVectorizer(binary=True)
        x_train = vectorizer.fit_transform(X_train)
        x_test = vectorizer.transform(X_test)
        pickle.dump(vectorizer,open(directoryname+"/Bayes_sparse_matrix_"+msg+str(id),"wb"))
        #trainieren
        naive_bayes.fit(x_train, y_train)
        predictions = naive_bayes.predict(x_test)
        with open(directoryname_output+"/Bayes_Pattern_Model_evaluation_"+msg+str(id),"w") as file:
            file.write("Accuracy score"+ str (accuracy_score(y_test, predictions))+"\n")
            file.write('Precision score: ' + str (precision_score(y_test, predictions,pos_label=0))+"\n")
            file.write('Recall score: ' + str (recall_score(y_test, predictions,pos_label=0))+"\n")
            file.write('f1_score score: ' + str (f1_score(y_test, predictions,pos_label=0))+"\n")
        pickle.dump(naive_bayes,open(directoryname+"/Bayes_Pattern_Model_"+msg+str(id),"wb"))
 
    
    
    

    
    
    